# 实战：定时获取天气预报

# 定时与邮件

第一是定时功能，即程序可以根据我们设定的时间自动爬取数据；

第二是通知功能，即程序可以把爬取到的数据结果以邮件的形式自动发送到我们的邮箱。

# 明确目标

自动爬取每日的天气，并定时把天气数据和穿衣提示发送到你的邮箱。

# 分析过程

程序分成三个功能块：【爬虫】+【邮件】+【定时】

对爬虫部分，我们比较熟悉；而对通知部分，选择的是用邮件来通知，我们将使用smtplib、email库来实现这一需求；对定时功能，有一个schedule，方便好用。

## 爬虫

百度搜索天气：http://www.weather.com.cn/weather/101280601.shtml

点击"右键"——"检查"——"Network"，刷新页面，点看第0个请求：
<img src="pics/network.png" width="40%">

数据放在HTML里，没问题。那我们点击Elements：
<img src="pics/element.png" width="40%">

温度数据放在<p class="tem">之下。

可以发现“多云”这个数据所在的位置：
    
“多云”所在的位置是<p title="多云" class="wea">多云</p>。

<img src="pics/目标数据.png" width="40%">

In [ ]:
import requests
from bs4 import BeautifulSoup
#引入requests库和BeautifulSoup库
headers={'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
#封装headers
url='http://www.weather.com.cn/weather/101280601.shtml'
#把URL链接赋值到变量url上。
res=requests.get(url,headers=headers)
#发送requests请求，并把响应的内容赋值到变量res中。
print(res.status_code)
#检查响应状态是否正常
print(res.text)
#打印出res对象的网页源代码

那我们在网页上点击"右键"——"查看网页源代码"，会弹出一个新的标签页，然后搜索charset，查看一下编码方式。

In [ ]:
import requests
from bs4 import BeautifulSoup
headers={'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
url='http://www.weather.com.cn/weather/101280601.shtml'
res=requests.get(url,headers=headers)
res.encoding='utf-8'
print(res.status_code)
print(res.text)

用BeautifulSoup模块解析和提取数据。

In [ ]:
import requests
from bs4 import BeautifulSoup
headers={'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
url='http://www.weather.com.cn/weather/101280601.shtml'
res=requests.get(url,headers=headers)
res.encoding='utf-8'
bsdata=BeautifulSoup(res.text,'html.parser')
#使用bs模块解析获取到的数据
data1= bsdata.find(class_='tem')
#使用find()取出天气的温度数据
data2= bsdata.find(class_='wea')
#使用find()取出天气的文字描述
print(data1.text)
#取出变量data1中的字符串内容，并打印
print(data2.text)
#取出变量data2中的字符串内容，并打印

## 邮件发送

In [ ]:
import smtplib 
from email.mime.text import MIMEText
from email.header import Header
#引入smtplib、MIMETex和Header

mailhost='smtp.qq.com'
#把qq邮箱的服务器地址赋值到变量mailhost上，地址应为字符串格式
qqmail = smtplib.SMTP()
#实例化一个smtplib模块里的SMTP类的对象，这样就可以调用SMTP对象的方法和属性了
qqmail.connect(mailhost,25)
#连接服务器，第一个参数是服务器地址，第二个参数是SMTP端口号。
#以上，皆为连接服务器。

account = input('请输入你的邮箱：')
#获取邮箱账号，为字符串格式
password = input('请输入你的密码：')
#获取邮箱密码，为字符串格式
qqmail.login(account,password)
#登录邮箱，第一个参数为邮箱账号，第二个参数为邮箱密码
#以上，皆为登录邮箱。

receiver=input('请输入收件人的邮箱：')
#获取收件人的邮箱。

content=input('请输入邮件正文：')
#输入你的邮件正文，为字符串格式
message = MIMEText(content, 'plain', 'utf-8')
#实例化一个MIMEText邮件对象，该对象需要写进三个参数，分别是邮件正文，文本格式和编码
subject = input('请输入你的邮件主题：')
#输入你的邮件主题，为字符串格式
message['Subject'] = Header(subject, 'utf-8')
#在等号的右边是实例化了一个Header邮件头对象，该对象需要写入两个参数，分别是邮件主题和编码，然后赋值给等号左边的变量message['Subject']。
#以上，为填写主题和正文。

try:
    qqmail.sendmail(account, receiver, message.as_string())
    print ('邮件发送成功')
except:
    print ('邮件发送失败')
qqmail.quit()
#以上为发送邮件和退出邮箱。

smtplib库主要负责的是横向的连接服务器、登录、发送和退出；而email库主要负责的是邮件主题和正文。

## 定时

Python有两个内置的标准库——time和datetime。

标准库一般意味着最原始最基础的功能，第三方库很多是去调用标准库中封装好了的操作函数。比如schedule，就是用time和datetime来实现的。

并不意味着time和datetime比schedule差，只是这个项目场景下，我们倾向于调用schedule。

In [ ]:
import schedule
import time
#引入schedule和time

def job():
    print("I'm working...")
#定义一个叫job的函数，函数的功能是打印'I'm working...'

schedule.every(10).minutes.do(job)       #部署每10分钟执行一次job()函数的任务
schedule.every().hour.do(job)            #部署每×小时执行一次job()函数的任务
schedule.every().day.at("10:30").do(job) #部署在每天的10:30执行job()函数的任务
schedule.every().monday.do(job)          #部署每个星期一执行job()函数的任务
schedule.every().wednesday.at("13:15").do(job)#部署每周三的13：15执行函数的任务

while True:
    schedule.run_pending()
    time.sleep(1)    
#13-15都是检查部署的情况，如果任务准备就绪，就开始执行任务。

# 代码组装

爬虫部分，封装前：

In [ ]:
import requests
from bs4 import BeautifulSoup
headers={'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
url='http://www.weather.com.cn/weather/101280601.shtml'
res=requests.get(url,headers=headers)
res.encoding='utf-8'
bsdata=BeautifulSoup(res.text,'html.parser')
data1= bsdata.find(class_='tem')
data2= bsdata.find(class_='wea')
print(data1.text)
print(data2.text)

In [ ]:
# 封装后
import requests
from bs4 import BeautifulSoup

def weather_spider():
    headers={'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
    url='http://www.weather.com.cn/weather/101280601.shtml'
    res=requests.get(url,headers=headers)
    res.encoding='utf-8'
    soup=BeautifulSoup(res.text,'html.parser')
    data1= soup.find(class_='tem')
    data2= soup.find(class_='wea')
    tem=data1.text
    weather=data2.text
    return tem,weather

In [ ]:
# 邮件封装前
from email.mime.text import MIMEText
from email.header import Header

mailhost='smtp.qq.com'
qqmail = smtplib.SMTP()
qqmail.connect(mailhost,25)

account = input('请输入你的邮箱：')
password = input('请输入你的密码：')
qqmail.login(account,password)

receiver=input('请输入收件人的邮箱：')

content=input('请输入邮件正文：')
message = MIMEText(content, 'plain', 'utf-8')
subject = input('请输入你的邮件主题：')
message['Subject'] = Header(subject, 'utf-8')

try:
    qqmail.sendmail(account, receiver, message.as_string())
    print ('邮件发送成功')
except:
    print ('邮件发送失败')
qqmail.quit()

In [ ]:
# 封装后
import smtplib
from email.mime.text import MIMEText
from email.header import Header

account = input('请输入你的邮箱：')
password = input('请输入你的密码：')
receiver = input('请输入收件人的邮箱：')

def send_email(tem,weather):
    mailhost='smtp.qq.com'
    qqmail = smtplib.SMTP()
    qqmail.connect(mailhost,25)
    qqmail.login(account,password)
    content= '亲爱的，今天的天气是：'+tem+weather
    message = MIMEText(content, 'plain', 'utf-8')
    subject = '今日天气预报'
    message['Subject'] = Header(subject, 'utf-8')
    try:
        qqmail.sendmail(account, receiver, message.as_string())
        print ('邮件发送成功')
    except:
        print ('邮件发送失败')
    qqmail.quit()

## 组装

In [ ]:
import requests
import smtplib
import schedule
import time
from bs4 import BeautifulSoup
from email.mime.text import MIMEText
from email.header import Header

account = input('请输入你的邮箱：')
password = input('请输入你的密码：')
receiver = input('请输入收件人的邮箱：')

def weather_spider():
    headers={'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
    url='http://www.weather.com.cn/weather/101280601.shtml'
    res=requests.get(url,headers=headers)
    res.encoding='utf-8'
    soup=BeautifulSoup(res.text,'html.parser')
    tem1= soup.find(class_='tem')
    weather1= soup.find(class_='wea')
    tem=tem1.text
    weather=weather1.text
    return tem,weather

def send_email(tem,weather):
    mailhost='smtp.qq.com'
    qqmail = smtplib.SMTP()
    qqmail.connect(mailhost,25)
    qqmail.login(account,password)
    content= tem+weather
    message = MIMEText(content, 'plain', 'utf-8')
    subject = '今日天气预报'
    message['Subject'] = Header(subject, 'utf-8')
    try:
        qqmail.sendmail(account, receiver, message.as_string())
        print ('邮件发送成功')
    except:
        print ('邮件发送失败')
    qqmail.quit()

def job():
    print('开始一次任务')
    tem,weather = weather_spider()
    send_email(tem,weather)
    print('任务完成')

schedule.every().day.at("07:30").do(job) 
while True:
    schedule.run_pending()
    time.sleep(1)

保持程序一直运行的状态，和电脑在一直开机的状态。因为如果程序结束或者电脑关机了的话，就不会定时爬取天气信息了。

在程序员真实的开发环境中，程序一般都会挂在远端服务器，因为远端服务器24小时都不会关机，就能保证定时功能的有效性了。如果你也想让程序挂在远端服务器的话，需要自己去做一些额外的学习。